In [1]:
import requests, pandas as pd
from bs4 import BeautifulSoup

In [2]:
def get_odd_target_indicator():
    url = 'https://unstats.un.org/sdgs/metadata'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        data = []

        panels_heading_default = soup.find_all('div', class_="panel panel-default")
        
        for panel in panels_heading_default:
            target_names = []
            odd = panel.find('h4').text.strip()
            targets = panel.find_all('h5')
            target_names.append([t.text.strip() for t in targets])
            ul_tags = panel.find_all('ul')
            for i in range(len(ul_tags)):
                indicators = ul_tags[i].find_all('small')
                for indicator in indicators:
                    indicator_text = ' '.join(indicator.stripped_strings)
                    if indicator_text.startswith("Indicator"):
                        parts = indicator_text.split()[1].split('.')
                        objectif = int(parts[0])
                        cible = parts[1] 
                        matching_target = next((target for target in target_names[0] if target.startswith(f'Target {objectif}.{cible}:')), None)
                        data.append({
                            'Objectif': odd.replace('Goal',''),
                            'Cible': matching_target.replace('Target',''),
                            'Indicateur': indicator_text.replace('Indicator','')
                        })
        df = pd.DataFrame(data)
        # Enregistrer le résultat en csv
        df.to_csv('../Other/odd_target_indicator.csv', index=False)
        return df

    else:
        print("La requête a échoué avec le code d'état:", response.status_code)
        return None